<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/Seyoung/%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **KeyBert로 리뷰 키워드 추출하기**

In [9]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.6 MB/s 
     |████████████████████████████████| 465 kB 74.5 MB/s 


In [ ]:
!pip install sentence_transformers

In [12]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Mecab
from konlpy.tag import Okt
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
data = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_spell_check.csv')
data.head()

,class,conversation,labelling,label_sentence,cleaned_sentence,spell_checked_conversation
0,직장 내 괴롭힘 대화,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,"1,2,1,2,1,2,1,2,1,2",1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...
1,기타 괴롭힘 대화,야 너 앞니 왜 그렇게 튀어나왔냐?\n 태어날때부터 그랬어 물어보지마.\n 아 그럼...,"1,2,1,2,1,2,1,2,1,1,1,2,1,1",1:야 너 앞니 왜 그렇게 튀어나왔냐?\n2:태어날때부터 그랬어 물어보지마.\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날때부터 그랬어 물어보지마.\n아 그럼 태...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...
2,갈취 대화,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n 미안 나 오늘은 진짜 돈이 없...,"1,2,1,2,1,2,1,2,1,2",1:원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n2:미안 나 오늘은 진짜 돈...,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n미안 나 오늘은 진짜 돈이 없어...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼?\n미안 나 오늘은 진짜 돈이...
3,갈취 대화,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,"1,2,1,2,1,2,1,2,1,2",1:너 저번에 술먹은 날 기억해?\n2:아니 왜?\n1:야 300만원만 가져와.\n...,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,너 저번에 술 먹은 날 기억해?\n아니 왜?\n야 300만 원만 가져와.\n뭔 소리...
4,협박 대화,너 또 내말 무시하냐?\n 이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 ...,"1,1,2,1,1,2,1,2,1,1",1:너 또 내말 무시하냐?\n1:이새끼 널 좆으로 보나본데?\n2:아냐 진짜 시간이...,너 또 내말 무시하냐?\n이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 미...,너 또 내말 무시하냐?\n이 새끼 널 좆으로 보나 본데?\n아냐 진짜 시간이 없었어...


In [5]:
tokenizer = Okt()

In [6]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [7]:
def read_token(Class):
    result = []
    for c in Class:
        tokenlist = tokenizer.pos(c, stem=True, norm=True)
        for word in tokenlist:
            if word[1] in ["Noun", "Verb"]:
                result.append(word[0])

    return ' '.join(result)

In [10]:
etc = data[data['class']=='기타 괴롭힘 대화'].reset_index(drop=True).spell_checked_conversation
etc_list = read_token(etc)

# tort = data[data['class']=='갈취 대화'].reset_index(drop=True).spell_checked_conversation
# extort_list = read_token(extort)

# company = data[data['class']=='직장 내 괴롭힘 대화'].reset_index(drop=True).spell_checked_conversation
# company_list = read_token(company)

# blackmaile = data[data['class']=='협박 대화'].reset_index(drop=True).spell_checked_conversation
# blackmaile_list = read_token(blackmaile)

# len(etc_list),len(extort_list), len(company_list), len(blackmaile_list)

In [11]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([etc_list])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 56601
trigram 다섯개만 출력 : ['가게 규정' '가게 규정 수가' '가게 나락' '가게 나락 가다' '가게 냄새']


In [12]:
genre = etc
doc = ''
for g in genre:
    doc = doc + g

doc[:10]

'야 너 앞니 왜 그'

In [13]:
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [14]:
top_n = 10
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [15]:
keywords

['갑자기 거지 새끼',
 '까불다 항상 놀리다',
 '그냥 장난 받아들이다',
 '별로 푸다 거지',
 '뒤지다 갑자기 화나다',
 '일부러 놀리다 거지',
 '놀리다 재미 보다',
 '편견 오해 거지',
 '깎다 그냥 받아들이다',
 '견우직녀 웃기 내놓다']

# **TextRank**

In [14]:
data.head()

,class,conversation,labelling,label_sentence,cleaned_sentence,spell_checked_conversation
0,직장 내 괴롭힘 대화,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,"1,2,1,2,1,2,1,2,1,2",1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...
1,기타 괴롭힘 대화,야 너 앞니 왜 그렇게 튀어나왔냐?\n 태어날때부터 그랬어 물어보지마.\n 아 그럼...,"1,2,1,2,1,2,1,2,1,1,1,2,1,1",1:야 너 앞니 왜 그렇게 튀어나왔냐?\n2:태어날때부터 그랬어 물어보지마.\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날때부터 그랬어 물어보지마.\n아 그럼 태...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...
2,갈취 대화,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n 미안 나 오늘은 진짜 돈이 없...,"1,2,1,2,1,2,1,2,1,2",1:원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n2:미안 나 오늘은 진짜 돈...,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n미안 나 오늘은 진짜 돈이 없어...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼?\n미안 나 오늘은 진짜 돈이...
3,갈취 대화,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,"1,2,1,2,1,2,1,2,1,2",1:너 저번에 술먹은 날 기억해?\n2:아니 왜?\n1:야 300만원만 가져와.\n...,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,너 저번에 술 먹은 날 기억해?\n아니 왜?\n야 300만 원만 가져와.\n뭔 소리...
4,협박 대화,너 또 내말 무시하냐?\n 이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 ...,"1,1,2,1,1,2,1,2,1,1",1:너 또 내말 무시하냐?\n1:이새끼 널 좆으로 보나본데?\n2:아냐 진짜 시간이...,너 또 내말 무시하냐?\n이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 미...,너 또 내말 무시하냐?\n이 새끼 널 좆으로 보나 본데?\n아냐 진짜 시간이 없었어...


In [15]:
etc = data[data['class']=='기타 괴롭힘 대화'].reset_index(drop=True).spell_checked_conversation

In [17]:
etc

0       야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...
1       오 신발 샀냐?\n응. 전에 신던 게 다 헤져서. 부모님이 사주셨어.\n너네 집 꽤...
2       영하 학생 이리 나와요\n네 선생님\n선생님이 왜 부른 줄 알아\n잘 모르겠습니다 ...
3       고객님 아이가 키가 안돼서 탑승이 어려울 것 같아요\n아니 우리애가 키가 좀 작아서...
4       저기요 아무도 안 계세요?\n네 손님! 무슨 일이세요?\n저 이거 환불할게요!\n손...
                              ...                        
1060    너는 이걸 그림이라고 그렸니?\n이리 줘!\n진짜 손재주라고는 전혀 없다.\n응 점...
1061    엄마 제가 물건 놔뒀는데 치웠어요\n아니 안 치웠는데\n제가 물건 여기 놔뒀는데 없...
1062    얘들아 공연 연습하자.\n이번 주에 최종 평가 있어. 빨리하자.\n야 김정미 너는 ...
1063    나를 왜 자꾸 때려\n때리고 싶으니깐.\n내가 뭐 잘못했어?\n아니 잘못 없지\n근...
1064    준하야 넌 대가리가 왜 이렇게 크냐?\n내 머리가 뭐.\n밥 먹으면 대가리만 크냐 ...
Name: spell_checked_conversation, Length: 1065, dtype: object

In [18]:
cleaned_sents = []
for e in etc:
    cleaned_sents.append(e)

In [3]:
!pwd

/content/drive/MyDrive


In [2]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [4]:
!ls

 노래	        아이펠톤			       Exploration_Data
'캐글 데이터'   미니아펠톤			       GoingDeeper_Data
 녹음	       '혼자공부하는 머신러닝'		       GoodNotes
 백준	       '아이펠(AIFFEL) 서류전형 지원서.gdoc'   miniAIFFELTHON_4.ipynb
 캡스톤         AIFFELTHON			       textrank
 데이터톤      'Colab Notebooks'


In [25]:
# KoNLPy Komoran 토크나이저

okt = Okt()
def komoran_tokenizer(sent):
    words = okt.pos(sent, stem=True, norm=True)
    words = [word[0] for word in words if word[1] in ["Noun", "Verb"]]
    return words

# TextRank Keyword Extraction
from textrank import KeywordSummarizer

keyword_summarizer = KeywordSummarizer(tokenize=komoran_tokenizer, min_count=2, min_cooccurrence=3)
keyword_summarizer.summarize(cleaned_sents, topk=20)

[('하다', 143.42643897066952),
 ('너', 66.85319877274607),
 ('거', 59.843623283520486),
 ('네', 46.06094503516861),
 ('내', 43.2188760148166),
 ('말', 37.768903274129116),
 ('안', 37.76551134547265),
 ('왜', 36.49569204018559),
 ('나', 34.0776070976641),
 ('보다', 33.47751569399467),
 ('뭐', 28.853871499307004),
 ('좀', 24.09967652716155),
 ('진짜', 21.32468724302968),
 ('이', 19.15376142345806),
 ('것', 17.17579833364496),
 ('우리', 16.94600274577093),
 ('알다', 16.774519675348767),
 ('오다', 14.996143030904438),
 ('되다', 14.977961817259896),
 ('마', 14.863322854745205)]

# **KR-WordRank**

In [27]:
%cd 아이펠톤

/content/drive/MyDrive/아이펠톤


In [28]:
!ls

 일반대화csv생성.ipynb		       '일반대화추가 성능확인.ipynb'
 daily.csv			        krwordrank
'data_TFIDF(Verb,띄어쓰기추가).ipynb'   T5-finetuning2.ipynb
 DKTCdata			        T5-finetuning3.ipynb
'idx_2650 - train.csv'		        textrank
 허깅페이스.ipynb


In [29]:
from krwordrank.sentence import summarize_with_sentences

In [40]:
tokenizer = Okt()

In [45]:
def read_token(Class):
    result = []
    for c in Class:
        tokenlist = tokenizer.pos(c, stem=True, norm=True)
        for word in tokenlist:
            if word[1] in ["Noun", "Verb"]:
                result.append(word[0])

    return result

In [46]:
etc = data[data['class']=='기타 괴롭힘 대화'].reset_index(drop=True).spell_checked_conversation
etc_list = read_token(etc)

In [48]:
texts = etc_list
keywords, sents = summarize_with_sentences(texts, num_keywords=100, num_keysents=10)

## 가해자, 피해자

In [51]:
sen = data['label_sentence']
attack = []
damaged = []
for i in range(3912):
    sen_tokens = sen[i].split('\n')
    for s in sen_tokens:
        if s[0] == str(1):
            attack.append(s[2:])
        else:
            damaged.append(s[2:])

In [52]:
len(attack), len(damaged)

(21285, 19143)

In [53]:
attack_list = read_token(attack)
damaged_list = read_token(damaged)

In [60]:
# 가해자
texts = attack_list
keywords, sents = summarize_with_sentences(texts, num_keywords=100, num_keysents=10)

In [61]:
keywords.keys()

dict_keys(['버리', '해주', '나오', '빌리', '모르', '다니', '내놓', '죽이', '해보', '가져', '생기', '들어', '헤어', '시키', '보내', '장애', '아이', '개인', '보이', '나가', '제대', '부모', '뒤지', '그래', '똑바', '핸드', '갈비', '점심', '만들', '선생', '보고', '남자', '말대', '어차', '거짓', '떨어', '얼마', '이르', '프로', '그만', '사오', '인터', '마지', '노트', '꼬박', '육아', '수행', '아스', '에어', '커뮤', '스트', '받아', '친구', '비트', '아메', '푸하', '오토', '개그', '립스', '손가', '우리', '만나', '머리', '조심', '뭐라', '기다', '사회', '꺼지', '돌아', '커피', '아파', '데이', '와이', '끝나', '주머', '때리', '아가', '오타', '비밀', '바이', '드리', '가정', '피아', '웃기', '가보', '쓰레', '알아', '아저', '여자', '아줌', '찾아', '할머', '전화', '교통', '지나', '가만', '뛰어', '어머', '차리', '책임'])

In [56]:
# 피해자
texts = damaged_list
keywords, sents = summarize_with_sentences(texts, num_keywords=100, num_keysents=10)

In [59]:
keywords.keys()

dict_keys(['이르', '살리', '드리', '해주', '모르', '헤어', '갑자', '가져', '돌려', '해보', '누구', '어머', '선생', '그만', '빌리', '프로젝', '비밀', '괴롭', '들어', '점심', '마스', '뭐라', '남자', '나오', '감사', '보내', '영수', '마지', '안녕', '왜그', '컨디', '스트', '노트', '아메', '막무', '개인', '머리', '성희', '싸이', '커플', '정신차', '아저', '협박', '시키', '핸드', '부모', '인터', '부르', '내려', '나가', '우리', '다른', '기다리', '아이', '최대', '지나', '근무', '겁니', '해드리', '사기', '때리', '죽이', '다니', '이제', '휴대', '일부', '말씀', '얼마', '제대', '떨어', '어리', '도대', '다녀', '편의', '사주', '일주', '만나', '어딨', '조심', '와이', '해오', '놀리', '이야', '사생', '이벤', '아파', '가보', '아버', '생기', '끝나', '올리', '여자', '다음', '돌아', '데이', '컴퓨', '그치', '따르', '버리', '여기'])